# How to:
* this is used to create the gen_build_costs table for SWITCH
* need to rerun this file for each new build year (2020, 2030, 2040, 2050)
* in order to do that:
    * need to update settings_TD to appropriate year
        - model_year and model_first_planning_year
        - other dictionary keys in settings_TD that have model year (demand_response_resources, settings_management)
    * need to update other inputs from the extra_inputs file (fix the year)
        - scenario_inputs, heat_load_shifting, emission_policies
* make sure output csv is named for the correct year (gets used in the SWITCH_gen_build_predetermined_and_cost notebook)

In [1]:
'''
Import based on imports found in the notebooks located here:
Schivley Greg, PowerGenome, (2022), GitHub repository, 
    https://github.com/PowerGenome/PowerGenome/tree/master/notebooks
'''

###
# module issue
import os
import sys
module_path = os.path.abspath(os.getcwd() + '\\..')
if module_path not in sys.path:
    sys.path.append(module_path)
###

import pandas as pd
import numpy as np

from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    check_settings
)
from powergenome.resource_clusters import ResourceGroup
from pathlib import Path

import pandas as pd
from powergenome.fuels import fuel_cost_table
from powergenome.generators import GeneratorClusters
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    check_settings
)
from powergenome.eia_opendata import fetch_fuel_prices
import geopandas as gpd
from powergenome.generators import *
from powergenome.external_data import (
    make_demand_response_profiles,
    make_generator_variability,
)

In [2]:
'''
Import based on imports found in the notebooks located here:
Schivley Greg, PowerGenome, (2022), GitHub repository, 
    https://github.com/PowerGenome/PowerGenome/tree/master/notebooks
'''

cwd = Path.cwd()

settings_path = (
    cwd / "settings_TD.yml" 
)
settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
scenario_definitions = pd.read_csv(
    settings["input_folder"] / settings["scenario_definitions_fn"]
)
scenario_settings = build_scenario_settings(settings, scenario_definitions)

pudl_engine, pudl_out, pg_engine = init_pudl_connection(
    freq="AS",
    start_year=min(settings.get("data_years")),
    end_year=max(settings.get("data_years")),
)

check_settings(settings, pg_engine)


    *****************************
    The ATB technology "LandbasedWind_Class1" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <tech>_<tech_detail> string might be included, but it is best practice to
    include the full name in this format. Check your settings file.
        

    *****************************
    The ATB technology "OffShoreWind_Class1" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <tech>_<tech_detail> string might be included, but it is best practice to
    include the full name in this format. Check your settings file.
        

    *****************************
    The ATB technology "UtilityPV_Class1" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <tec

In [3]:
'''
Schivley Greg, PowerGenome, (2022), GitHub repository, 
https://github.com/PowerGenome/PowerGenome/blob/master/notebooks/Existing%20and%20new%20generators.ipynb
'''
# Change year
gc = GeneratorClusters(pudl_engine, pudl_out, pg_engine, scenario_settings[2050]["p1"])

C:\Users\dawso\miniconda3\envs\powergenome\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\dawso\miniconda3\envs\powergenome\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


210.70000000000002  MW without lat/lon


In [4]:
'''
Schivley Greg, PowerGenome, (2022), GitHub repository, 
https://github.com/PowerGenome/PowerGenome/blob/master/notebooks/Existing%20and%20new%20generators.ipynb
'''
# adjust settings_TD to make atb
new_gen_decade = gc.create_new_generators()

No model tag values found for LDS ('LDS')
No model tag values found for MinCapTag_2 ('MinCapTag_2')
No model tag values found for MinCapTag_3 ('MinCapTag_3')
No model tag values found for Reg_Max ('Reg_Max')
No model tag values found for Rsv_Max ('Rsv_Max')
C:\Users\dawso\0Capstone\no_cluster3\PowerGenome\powergenome\external_data.py:142: FutureWarning: The default value of regex will change from True to False in a future version.
  new_resource_df["technology"]
Transmission investment costs are missing or zero for some resources and will not be included in the total investment costs.


In [5]:
# read in csv outputs from PG
# existing_gen = pd.read_csv('PG_output_csv/existing_gen_WECC.csv', index_col=0)
new_gen = pd.read_csv('PG_output_csv/new_gen_WECC.csv', index_col=0)
#existing_variability = pd.read_csv('PG_output_csv/5existing_variability_WECC.csv', index_col=0)
# potential_build_yr = pd.read_csv('PG_output_csv/gc_units_model_WECC.csv', index_col=0)
# all_gen = pd.read_csv('PG_output_csv/all_gen_WECC.csv', index_col=0)
# fuels = pd.read_csv('PG_output_csv/fuels_WECC.csv', index_col=0)
# fuel_prices = pd.read_csv('PG_output_csv/fuel_prices_WECC.csv', index_col=0)
#load_curves = pd.read_csv('PG_output_csv/load_curves_WECC.csv', index_col=0)

In [6]:
new_gen # check length against new_gen_decade

,technology,basis_year,Fixed_OM_Cost_per_MWyr,Fixed_OM_Cost_per_MWhyr,Var_OM_Cost_per_MWh,capex_mw,capex_mwh,Inv_Cost_per_MWyr,Inv_Cost_per_MWhyr,Heat_Rate_MMBTU_per_MWh,...,Reg_Max,Rsv_Max,cluster,spur_miles,interconnect_annuity,interconnect_capex_mw,spur_capex,spur_inv_mwyr,plant_inv_cost_mwyr,Resource
0,NaturalGas_CCCCSAvgCF_Conservative,2020.0,62416,0,5.493818,2.543010e+06,0.000000,212995.048927,0,7.159,...,0,0,1.0,20,0.0,0,175500,8501.048927,204494,naturalgas_ccccsavgcf_conservative
1,NaturalGas_CCAvgCF_Moderate,2020.0,26174,0,1.668280,1.006061e+06,0.000000,121406.000000,0,6.363,...,0,0,1.0,0,0.0,0,0,0.000000,121406,naturalgas_ccavgcf_moderate
2,NaturalGas_CTAvgCF_Moderate,2020.0,20038,0,4.736380,8.893702e+05,0.000000,97041.000000,0,9.717,...,0,0,1.0,0,0.0,0,0,0.000000,97041,naturalgas_ctavgcf_moderate
3,Battery_*_Moderate,2020.0,6155,6630,0.000000,2.462011e+05,265222.620516,21072.000000,22700,0.000,...,0,0,1.0,0,0.0,0,0,0.000000,21072,battery_moderate
4,NaturalGas_CCS100_Moderate,2020.0,72086,0,5.911348,2.659010e+06,0.000000,222323.048927,0,7.524,...,0,0,1.0,20,0.0,0,175500,8501.048927,213822,naturalgas_ccs100_moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,NaturalGas_CCCCSAvgCF_Conservative,2020.0,62416,0,5.493818,2.543010e+06,0.000000,170411.000000,0,7.159,...,0,0,1.0,0,0.0,0,0,0.000000,170411,naturalgas_ccccsavgcf_conservative
76,NaturalGas_CCAvgCF_Moderate,2020.0,26174,0,1.668280,1.006061e+06,0.000000,84262.000000,0,6.363,...,0,0,1.0,0,0.0,0,0,0.000000,84262,naturalgas_ccavgcf_moderate
77,NaturalGas_CTAvgCF_Moderate,2020.0,20038,0,4.736380,8.893702e+05,0.000000,74248.000000,0,9.717,...,0,0,1.0,0,0.0,0,0,0.000000,74248,naturalgas_ctavgcf_moderate
78,Battery_*_Moderate,2020.0,6155,6630,0.000000,2.462011e+05,265222.620516,20867.000000,22480,0.000,...,0,0,1.0,0,0.0,0,0,0.000000,20867,battery_moderate


In [7]:
new_gen_decade

,technology,basis_year,Fixed_OM_Cost_per_MWyr,Fixed_OM_Cost_per_MWhyr,Var_OM_Cost_per_MWh,capex_mw,capex_mwh,Inv_Cost_per_MWyr,Inv_Cost_per_MWhyr,Heat_Rate_MMBTU_per_MWh,...,Reg_Max,Rsv_Max,cluster,spur_miles,interconnect_annuity,interconnect_capex_mw,spur_capex,spur_inv_mwyr,plant_inv_cost_mwyr,Resource
0,NaturalGas_CCCCSAvgCF_Conservative,2050.0,62416,0,5.493818,1.962712e+06,0.000000,166331.048927,0,7.159,...,0,0,1.0,20,0.0,0,175500,8501.048927,157830,naturalgas_ccccsavgcf_conservative
1,NaturalGas_CCAvgCF_Moderate,2050.0,26174,0,1.668280,8.693962e+05,0.000000,104914.000000,0,6.363,...,0,0,1.0,0,0.0,0,0,0.000000,104914,naturalgas_ccavgcf_moderate
2,NaturalGas_CTAvgCF_Moderate,2050.0,20038,0,4.736380,7.211678e+05,0.000000,78688.000000,0,9.717,...,0,0,1.0,0,0.0,0,0,0.000000,78688,naturalgas_ctavgcf_moderate
3,Battery_*_Moderate,2050.0,4349,2436,0.000000,1.739989e+05,97458.020465,14745.000000,8259,0.000,...,0,0,1.0,0,0.0,0,0,0.000000,14745,battery_moderate
4,NaturalGas_CCS100_Moderate,2050.0,72086,0,5.911348,2.078712e+06,0.000000,175659.048927,0,7.524,...,0,0,1.0,20,0.0,0,175500,8501.048927,167158,naturalgas_ccs100_moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,NaturalGas_CCCCSAvgCF_Conservative,2050.0,62416,0,5.493818,1.962712e+06,0.000000,131525.000000,0,7.159,...,0,0,1.0,0,0.0,0,0,0.000000,131525,naturalgas_ccccsavgcf_conservative
76,NaturalGas_CCAvgCF_Moderate,2050.0,26174,0,1.668280,8.693962e+05,0.000000,72816.000000,0,6.363,...,0,0,1.0,0,0.0,0,0,0.000000,72816,naturalgas_ccavgcf_moderate
77,NaturalGas_CTAvgCF_Moderate,2050.0,20038,0,4.736380,7.211678e+05,0.000000,60206.000000,0,9.717,...,0,0,1.0,0,0.0,0,0,0.000000,60206,naturalgas_ctavgcf_moderate
78,Battery_*_Moderate,2050.0,4349,2436,0.000000,1.739989e+05,97458.020465,14602.000000,8178,0.000,...,0,0,1.0,0,0.0,0,0,0.000000,14602,battery_moderate


In [8]:
new_gen_decade.to_csv('new_gen_2050.csv')